In [1]:
import time
import numpy as np
from scipy import sparse
import pandas as pd
import h5py
import anndata

from dredFISH.Analysis import basicu

In [10]:
res_dir = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/'

In [3]:
# ddir = '/bigstore/binfo/mouse/Brain/Sequencing/Allen_10X_CtxHippo/source/'
ddir = '/bigstore/binfo/mouse/Brain/Sequencing/Allen_SmartSeq_CtxHippo/source/'

# load metadata

In [4]:
f = ddir + 'metadata.csv'
df1 = pd.read_csv(f, low_memory=False).set_index('sample_name')
f = ddir + 'tsne.csv'
df2 = pd.read_csv(f, low_memory=False).set_index('sample_name')
df = df1.join(df2) #.astype({'cluster_order': str})

# load data matrix 
Unpack the `hdf5` file:
- `data` 
    - `exon` (csc_matrix)
    - `t_exon` (csc_matrix)
    - `total_exon_counts` (1d array)
- `gene_names`  (1d array)
- `sample_names`  (1d array)


csc_matrix: dims, i, p, x
scipy
`csc_matrix((data, indices, indptr), [shape=(M, N)])`

In [5]:
ti = time.time()
f = ddir + 'expression_matrix.hdf5'
with h5py.File(f, 'r') as fh:
    print(fh.keys())
    print(fh['data'].keys())
    print(fh['data']['exon'].keys())
    
    dat = fh['data']['exon']
    dims = dat['dims'][()]
    print(dims)

    i = dat['i'][()]
    p = dat['p'][()]
    x = dat['x'][()]
    mat = sparse.csc_matrix((x,i,p), shape=dims)
    print(mat.shape)
    
    genes = fh['gene_names'][()]
    genes = np.array([gene.decode() for gene in genes])
    
    cells = fh['sample_names'][()] 
    cells = np.array([cell.decode() for cell in cells])

<KeysViewHDF5 ['data', 'gene_names', 'sample_names']>
<KeysViewHDF5 ['exon', 't_exon', 'total_exon_counts']>
<KeysViewHDF5 ['dims', 'i', 'p', 'x']>
[73363 45768]
(73363, 45768)


In [6]:
cells.shape, df.shape, np.intersect1d(cells, df.index.values).shape

((73363,), (73347, 58), (73347,))

In [7]:
# for each cell in meta, where it is in the matrix
cell_idx = basicu.get_index_from_array(cells, df.index.values)
(cell_idx == -1).sum()

0

In [8]:
adata = anndata.AnnData(mat.tocsr()[cell_idx,:],  # mat -- csr: easy for row(cell) selection
                   var=pd.DataFrame(index=genes), 
                   obs=df)
adata

/tmp/ipykernel_1270/104210243.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = anndata.AnnData(mat.tocsr()[cell_idx,:],  # mat -- csr: easy for row(cell) selection


AnnData object with n_obs × n_vars = 73347 × 45768
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'ce

In [11]:
dataid = 'scrna_ss_ctxhippo_a'
output = f'{res_dir}{dataid}_exon_count_matrix_v5.h5ad'
print(output)
adata.write(output)
!chmod 444 $output

/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v5.h5ad


# Done and check

In [12]:
# file = f'{res_dir}{dataid}_exon_count_matrix_v5.h5ad'
file = output
adata2 = anndata.read(file, backed='r')
adata2

AnnData object with n_obs × n_vars = 73347 × 45768 backed at '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v5.h5ad'
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injec